# Sentiment Analysis con KNN
## Clasificador en C++ 💪💪
Vamos a probar a nuestro bichito


Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [13]:
!cd .. && git submodule init
!cd .. && git submodule update
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python3)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

mkdir: cannot create directory ‘build’: File exists
-- The C compiler identification is GNU 7.4.0
-- The CXX compiler identification is GNU 7.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp: /usr/bin/python3 (found version "3.6.8") 
-- Found PythonLibs: /usr/lib/x86_64-linux-gnu/libpython3.6m.so
-- pybind11 v2.3.dev0
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/home2/clinux01/Metodos-TP2
-- Configuring done
-- Generating done
-- Build files have b

In [1]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python3 --version
import sentiment


/home2/clinux01/Metodos-TP2/notebooks
Python 3.6.8


In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd

!cd ../data && tar -xvf *.tgz

df = pd.read_csv("../data/imdb_small.csv", index_col=0)

print("Cantidad de documentos: {}".format(df.shape[0]))

tar: *.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
Cantidad de documentos: 12500


In [3]:
df.describe()

,type,review,label,file
count,12500,12500,12500,12500
unique,2,12469,2,12085
top,test,This show comes up with interesting locations ...,neg,5315_4.txt
freq,6275,2,6322,2


In [4]:
text_train = df[df.type == 'train']["review"]
label_train = df[df.type == 'train']["label"]

text_test = df[df.type == 'test']["review"]
label_test = df[df.type == 'test']["label"]

print("Cantidad de instancias de entrenamiento = {}".format(len(text_train)))
print("Cantidad de instancias de test = {}".format(len(text_test)))


Cantidad de instancias de entrenamiento = 6225
Cantidad de instancias de test = 6275


In [5]:
print("Class balance : {} pos {} neg".format(
    (label_train == 'pos').sum(),
    (label_train == 'neg').sum() ))


Class balance : 3081 pos 3144 neg


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.90, min_df=0.01, max_features=500)

vectorizer.fit(text_train)

X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values

In [7]:
# Esta cell convierte los casos de test y de entrenamiento
# al formato que requiere nuestro Knn y PCA

y_test2 = []
for i in range(len(y_test)):
    y_test2.insert(i,0)

for i in range(len(y_test)):
    if y_test[i] == True:
        y_test2[i] = 1
    else :
        y_test2[i] = 0
        
y_train2 = []
for i in range(len(y_train)):
    y_train2.insert(i,0)

for i in range(len(y_train)):
    if y_train[i] == True:
        y_train2[i] = 1
    else :
        y_train2[i] = 0



In [9]:
%%time

import sentiment
from sklearn.metrics import accuracy_score

from sentiment import PCA, KNNClassifier

#Experimentacion del modelo sin PCA variando el k
#print("k,accuracy")
#Experimentacion del modelo variando k y alpha
print("k,alpha,accuracy")
#for alpha in range(1,100,20): 
pca = PCA(61)
pca.fit(X_train.toarray())
X_train2 = pca.transform(X_train)
X_test2 = pca.transform(X_test)

clf = sentiment.KNNClassifier(76)
clf.fit(X_train2,y_train)
y_pred = clf.predict(X_test2)
        #print(y_pred)
res = 0
for i in range(len(y_test2)):
    if abs(y_test2[i] - y_pred[i])<1e-4:
        res =res + 1
        #acc = accuracy_score(y_test2, y_pred)
print("{},{},{}".format(76,61,res/len(y_pred)))


k,alpha,accuracy
76,61,0.6532270916334662
CPU times: user 1min, sys: 180 ms, total: 1min
Wall time: 1min


In [17]:
%%time

from sentiment import PCA, KNNClassifier

# Ejecucion con los valores optimos encontrados
# Correr con todos los casos de train y test
alphaOptimo = 80
kOptimo = 161

# PCA
pca = PCA(alphaOptimo)
pca.fit(X_train.toarray())
X_train2 = pca.transform(X_train)
X_test2 = pca.transform(X_test)

# Knn
clf = sentiment.KNNClassifier(kOptimo)
clf.fit(X_train2,y_train)

from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test2)

acc = accuracy_score(y_test, y_pred)
print("Accuracy: {}".format(acc))

Accuracy: 0.6543426294820717
CPU times: user 1min 11s, sys: 136 ms, total: 1min 11s
Wall time: 1min 11s


In [ ]:
%%time

import sentiment
from sklearn.metrics import accuracy_score

from sentiment import PCA, KNNClassifier

pca = PCA(61)
pca.fit(X_train.toarray())
X_train2 = pca.transform(X_train)
X_test2 = pca.transform(X_test)

clf = sentiment.KNNClassifier(76)
clf.fit(X_train2,y_train)
y_pred = clf.predict(X_test2)
        #print(y_pred)
res = 0
for i in range(len(y_test2)):
    if abs(y_test2[i] - y_pred[i])<1e-4:
        res = res + 1
        #acc = accuracy_score(y_test2, y_pred)
print("{},{},{}".format(161,alpha,res/len(y_pred)))